### Семинар 3. Классы. Введение

<br />

##### Мой первый класс

Реализуем класс - строку на С++.

Важнейший принцип языка, основа его основ - RAII (Resource Acquisition Is Initialization)

```c++
class String
{
public:
    String();
    String(const char *s);
    String(const char *s, int size);
    ~String();
    String(const String& rhs);
    String(String&& rhs);

    String& operator = (const String& rhs);
    String& operator = (String&& rhs);

    friend String operator + (const String& lhs, const String& rhs);

private:
    char* s_;  // pointer to null-terminated characters
    size_t l_; // strlen(s_) == l_
};
```

Для начала реализуем оператор сложения двух строк

```c++
String operator + (const String& lhs, const String& rhs)
{
    const size_t res_size = lhs.l_ + rhs.l_;
    const char* res_s = new char[res_size + 1];
    strcpy(res_s, lhs.s_);
    strncpy(res_s + lhs.l_, rhs.s_, rhs.l_);
    
    String s(res_s, res_size);

    delete[] res_s;
    return s;
}
```

Реализуем конструкторы

```c++
String::String()
    : s_(new char[1])
    , l_(0)
{
    s_[0] = 0;
}

String::String(const char* s)
{
    l_ = strlen(s);
    s_ = new char[l + 1];
    strcpy(s_, s);
}

String::String(const char *s, int size)
    : s_(new char[size + 1])
    , l_(size + 1)
{
    strncpy(s_, s, size);
    s_[l_] = 0;
}
```

Реализуем деструктор - код, который будет вызываться при уничтожении объекта

```c++
String::~String()
{
    delete[] s_;
}
```

Остановимся на секунду и всомним про RAII

Реализуем конструктор копирования

```c++
String::String(const String& rhs)
{
    s_ = new char[rhs.l_ + 1];
    l_ = rhs.l_;
    strcpy(s_, rhs.s_);
}
```

А теперь конструктор перемещения

```c++
String::String(String&& rhs)
    : s_(rhs.s_)
    , l_(rhs.l_)
{
}
```

Что-то пошло не так. Что именно?

Не работает такой код, а должен:

```c++
{
    String s1 = "run, Forest, run!";
    String s2 = std::move(s1);
}
```

Реализуем правильный конструктор перемещения

```c++
String::String(String&& rhs)
    : s_(rhs.s_)
    , l_(rhs.l_)
{
    rhs.s_ = 0;
    rhs.l_ = 0;
}
```

А точно ли он правильный?

Спойлер: rhs сломан

Третья попытка реализовать правильный конструктор перемещения

```c++
String::String(String&& rhs)
{
    s_ = rhs.s_;
    l_ = rhs.l_;

    rhs.s_ = new char[1];
    rhs.s_[0] = 0;
    rhs.l_ = 0;
}
```

Теперь реализуем оператор копирующего присваивания:

```c++
String& String::operator =(const String& rhs)
{
    delete[] s_;

    s_ = new char[rhs.l_ + 1];
    l_ = rhs.l_;
    strcpy(s_, rhs.s_);

    return *this;
}
```

Где ошибка?

Правильная реализация будет выглядеть так:

```c++
String& String::operator =(const String& rhs)
{
    if (this != &rhs)
    {
        delete[] s_;

        s_ = new char[rhs.l_ + 1];
        l_ = rhs.l_;
        strcpy(s_, rhs.s_);
    }
    return *this;
}
```

Или нет?

И оператор перемещающего присваивания:

```c++
String& String::operator =(String&& rhs)
{
    if (this != &rhs)
    {
        delete[] s_;

        s_ = rhs.s_;
        l_ = rhs.l_;

        rhs.s_ = new char[1];
        rhs.s_[0] = 0;
        rhs.l_ = 0;
    }

    return *this;
}
```

Замечания:
* Методы перемещения && плохи, они кидают исключения, лучше реализовать класс таким образом, чтобы методы перемещения не кидали исключений и объявить их noexcept. Как это сделать?
* Деструктор должен быть noexcept

<br />

##### Порядок конструирования

In [ ]:
# %load 1_ctors_order.cpp
#include <iostream>


class Name
{
public:
	Name() { std::cout << "Name "; }
	~Name() { std::cout << "~Name "; }
};

class Leg
{
public:
	Leg() { std::cout << "Leg "; }
	~Leg() { std::cout << "~Leg "; }
};

class Tail
{
public:
	Tail() { std::cout << "Tail "; }
	~Tail() { std::cout << "~Tail "; }
};

class Animal
{
public:
	Animal() { std::cout << "Animal "; }
	~Animal() { std::cout << "~Animal "; }

private:
	Name name_;
};

class Cat : public Animal
{
public:
	Cat() { std::cout << "Cat "; }
	~Cat() { std::cout << "~Cat "; }

private:
	Leg l1_;
	Leg l2_;
	Leg l3_;
	Leg l4_;
	Tail tail_;
};


// show this later
// Animal animal;

int main()
{
	std::cout << "hello!" << std::endl;
	Cat cat;
	std::cout << std::endl << "goodbye!" << std::endl;
	return 0;
}


Порядок вызова конструкторов:
1. сначала базовый класс
2. потом члены в порядке их объявления
3. потом конструктор самого класса

Порядок вызова деструкторов:
* обратно порядку вызова конструкторов

Скомпилируем и запустим

In [7]:
!clang++ -O3 1_ctors_order.cpp && ./a.out && rm -f a.out

hello!
Name Animal Leg Leg Leg Leg Tail Cat 
goodbye!
~Cat ~Tail ~Leg ~Leg ~Leg ~Leg ~Animal ~Name 